In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from make_filament_mesh import MakeFilamentMesh, WriteVtk
from pydrake.all import (AddContactMaterial, AddMultibodyPlant,
                         AddRigidHydroelasticProperties, Box, Capsule,
                         ConstantValueSource, ConstantVectorSource, Convex,
                         CoulombFriction, Cylinder, DeformableBodyConfig,
                         DiagramBuilder,
                         DifferentialInverseKinematicsIntegrator,
                         DifferentialInverseKinematicsParameters,
                         DiscreteDerivative, Filament, FixedOffsetFrame,
                         GeometryInstance, IllustrationProperties, Integrator,
                         LeafSystem, Mesh, Meshcat, MeshcatVisualizer,
                         MeshcatVisualizerParams, ModelInstanceIndex,
                         MultibodyPlant, MultibodyPlantConfig, Multiplexer,
                         Parser, PdControllerGains, PidController,
                         PiecewisePolynomial, PiecewisePose,
                         PiecewiseQuaternionSlerp, ProximityProperties,
                         RevoluteJoint, Rgba, RigidTransform, RotationMatrix,
                         SceneGraphConfig, Simulator, SpatialInertia, Sphere,
                         Value, VectorLogSink)

In [ ]:
meshcat = Meshcat()
meshcat.SetProperty("/Grid", "visible", False)
meshcat.SetProperty("/Axes", "visible", False)
meshcat.SetCameraPose([0, -0.1, 0.2], [0, 0, 0])

In [ ]:
def AddRoller(plant, name, position,
              radius=0.03, length=0.08, density=1000,
              friction_coeff=0.2,
              color=Rgba(0.4,0.4,0.9,1), crowned=False):
    roller = plant.AddRigidBody(
        name,
        SpatialInertia.SolidCylinderWithDensity(density, radius, length, [0,0,1])
    )
    offset_frame = plant.AddFrame(FixedOffsetFrame(
        f"world_offset_frame_to_{name}",
        plant.world_body(),
        RigidTransform(RotationMatrix.MakeXRotation(np.pi/2), position)
    ))
    joint = plant.AddJoint(RevoluteJoint(
        name=f"{name}_revolute_joint",
        frame_on_parent=offset_frame,
        frame_on_child=roller.body_frame(),
        axis=[0,0,1]
    ))
    proximity_props = ProximityProperties()
    AddContactMaterial(properties=proximity_props, friction=CoulombFriction(0.2,0.2))
    AddRigidHydroelasticProperties(properties=proximity_props, resolution_hint=radius * 0.4)
    if not crowned:
        shape = Cylinder(radius, length)
    else:
        shape = Convex("./crowned_pulley.vtk", [radius*2, radius*2, length])
    plant.RegisterCollisionGeometry(
        roller, RigidTransform(), shape, name, proximity_props
    )
    plant.RegisterVisualGeometry(
        roller, RigidTransform(), shape, name, color.rgba
    )
    plant.RegisterVisualGeometry(
        roller, RigidTransform([radius/2, 0, 0]),
        Box(radius * (0.95 if not crowned else 0.75), radius * 0.1, length * 1.001),
        f"{name}_indicator", [1,1,1,1]
    )
    return roller, joint

In [ ]:
builder = DiagramBuilder()

plant_config = MultibodyPlantConfig()
plant_config.time_step = 0.2e-3

scene_graph_config = SceneGraphConfig()
scene_graph_config.default_proximity_properties.compliance_type = "rigid"

plant, scene_graph = AddMultibodyPlant(plant_config, scene_graph_config, builder)

# Add rollers
roller_radius = 0.03
belt_radius = roller_radius + 0.001
roller_distance = np.pi * belt_radius * 1.5
roller1, joint1 = AddRoller(plant, "roller1", position=[-roller_distance/2,0,0], radius=roller_radius, crowned=True)
roller2, joint2 = AddRoller(plant, "roller2", position=[roller_distance/2,0,0], radius=roller_radius)
joint1_actuator = plant.AddJointActuator("roller1_actuator", joint1)
joint1_actuator.set_controller_gains(PdControllerGains(p=10, d=1))

# Add the belt
deformable_model = plant.mutable_deformable_model()

delta_theta = np.pi * 0.1
delta_x = delta_theta * belt_radius
belt_node_position = []
y = -0.02
for theta in np.arange(np.pi/2, np.pi*3/2, delta_theta):
    belt_node_position.append([belt_radius * np.cos(theta) - roller_distance/2, y, belt_radius * np.sin(theta)])
for x in np.arange(0.0, roller_distance, delta_x) - roller_distance/2:
    belt_node_position.append([x, y, -belt_radius])
for theta in np.arange(-np.pi/2, np.pi/2, delta_theta):
    belt_node_position.append([belt_radius * np.cos(theta) + roller_distance/2, y, belt_radius * np.sin(theta)])
for x in np.arange(roller_distance, 0.0, -delta_x) - roller_distance/2:
    belt_node_position.append([x, y, belt_radius])

belt = Filament(
    closed=True,
    node_pos=np.array(belt_node_position).T,
    cross_section=Filament.RectangularCrossSection(width=0.02, height=0.002),
    first_edge_m1=[0,1,0]
)
verts, elems = MakeFilamentMesh(belt, 0.005)

delta_x = delta_x * 0.95
delta_theta = 2 * np.pi / len(belt_node_position)
r = delta_x/2 / np.sin(delta_theta/2)
theta = np.arange(0, 2*np.pi, delta_theta)
belt_node_position = np.vstack([
    r * np.cos(theta),
    np.zeros_like(theta),
    r * np.sin(theta)
])
belt = Filament(
    closed=True,
    node_pos=belt_node_position,
    cross_section=Filament.RectangularCrossSection(width=0.02, height=0.002),
    first_edge_m1=[0,1,0]
)
verts2, elems2 = MakeFilamentMesh(belt, 0.005)
WriteVtk("belt.vtk", verts2, elems2)

geometry = GeometryInstance(RigidTransform(), Mesh("belt.vtk"), f"belt")

illus_props = IllustrationProperties()
illus_props.AddProperty("phong", "diffuse", Rgba(0.6,0.6,0.4,1.0))
geometry.set_illustration_properties(illus_props)

proximity_props = ProximityProperties()
AddContactMaterial(
    properties=proximity_props,
    friction=CoulombFriction(0.6,0.6),
)
geometry.set_proximity_properties(proximity_props)

config = DeformableBodyConfig()
config.set_youngs_modulus(6.2e5)
config.set_poissons_ratio(0.4999)
config.set_mass_density(1155)
config.set_mass_damping_coefficient(1.1)

unused_resolution_hint = 9999
belt_body_id = deformable_model.RegisterDeformableBody(
    geometry, config, unused_resolution_hint)
belt_body = deformable_model.GetBody(belt_body_id)

# Finalize the multibody plant
plant.Finalize()

# Add constant speed source
speed_source = builder.AddSystem(ConstantVectorSource([np.pi]))
position_source = builder.AddSystem(Integrator(1))
builder.Connect(speed_source.get_output_port(), position_source.get_input_port())
mux = builder.AddSystem(Multiplexer(2))
builder.Connect(position_source.get_output_port(), mux.get_input_port(0))
builder.Connect(speed_source.get_output_port(), mux.get_input_port(1))
builder.Connect(
    mux.get_output_port(),
    plant.get_desired_state_input_port(ModelInstanceIndex(1))
)

# Add Meshcat visualizer
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat, MeshcatVisualizerParams()
)

# Build and simulate
diagram = builder.Build()
simulator = Simulator(diagram)

context = simulator.get_context()
plant_context = diagram.GetSubsystemContext(plant, context)
belt_body.SetPositions(plant_context, verts)

simulator.Initialize()
simulator.set_target_realtime_rate(1.0)

visualizer.StartRecording()
simulator.AdvanceTo(60.0)
visualizer.StopRecording()
visualizer.PublishRecording()